In [ ]:
!git clone https://github.com/jherberg462/SIIM-ISIC.git --depth 1

In [ ]:
import numpy as np 
import pandas as pd 


import os

import tensorflow as tf
from tensorflow.keras import layers 
from tensorflow import keras
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

try:
    from kaggle_datasets import KaggleDatasets
    dataset_gcs = KaggleDatasets().get_gcs_path('siim-isic-melanoma-classification')
    print('got GCS path via KaggleDatasets .get_gcs_path method')
except ModuleNotFoundError:
    #hardcode path when running outside of Kaggle
    dataset_gcs = 'gs://kds-1e4e2af3c4717d1070d563d5555e3e09bd7df325fccbfa453ee9b532'

import sys
sys.path.insert(0, './SIIM-ISIC')
from input_pipeline import (decode_image_label, decode_image, 
                            normalize_image_label, random_flip, 
                            get_train_ds, get_test_ds, get_ds_size)
from params import params

In [ ]:
tf.__version__


In [ ]:
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
params['batch_size'] = params['batch_size'] * strategy.num_replicas_in_sync


In [ ]:
train_df = pd.read_csv(dataset_gcs + '/train.csv')
train_df.groupby('target').count()

In [ ]:
def create_model_pretrained(input_shape=[*params['img_size'], 3], bias_output=None):
    '''
    function to create a model that will be used to train DS via xfer learning
    
    args:
        input_shape: array, default: [1024, 1024, 3], shape
            of input tensor that will be fed into model
    
    returns:
        model: keras.Model() model
    
    '''
    
    input_tensor = layers.Input(shape=input_shape, name='images_input')
    

    
    #s/b 299, 299, 3 // needs to be -1 --> 1
    xception = tf.keras.applications.Xception(include_top=False,
                                                       input_shape=input_shape,
                                                       classes=1)(input_tensor)
    end_of_xception = layers.GlobalAveragePooling2D()(xception)
    dense_layers = layers.Dense(8)(end_of_xception)
    
    



    
    if bias_output is not None:
        bias_output = tf.keras.initializers.Constant(bias_output)
    output = layers.Dense(1, activation='sigmoid', bias_initializer=bias_output)(dense_layers)
    model = keras.Model(inputs=input_tensor, outputs=output)
    
    
    metrics = [
          keras.metrics.TruePositives(name='tp'),
          keras.metrics.FalseNegatives(name='fn'), 
          keras.metrics.BinaryAccuracy(name='accuracy'),
          keras.metrics.AUC(name='auc'),
    ]
    schedule = None
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(),
        loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.01),
        metrics=metrics)
    
    
    return model

In [ ]:
#get test file paths
test_files = tf.io.gfile.glob(dataset_gcs + '/tfrecords/test*.tfrec')

#get train and validation file paths
train_files, valid_files = train_test_split(tf.io.gfile.glob(dataset_gcs + '/tfrecords/train*.tfrec'),
                              test_size=.1, random_state=1)

#create datasets
train_ds = get_train_ds(train_files, params['batch_size'])
valid_ds = get_train_ds(valid_files, params['batch_size'])
test_ds = get_test_ds(test_files, params['batch_size'])

In [ ]:
train_size, valid_size = get_ds_size(train_files), get_ds_size(valid_files)
test_size = get_ds_size(test_files)
print('the dataset consists of: {} training images, {} validation images, and {} test images'.
     format(train_size, valid_size, test_size))

In [ ]:
epoch_steps = train_size / params['batch_size'] 
valid_steps = valid_size / params['batch_size']
test_steps = 1.0 * test_size / params['batch_size']

In [ ]:
#calculate class weights

targets = train_df.groupby('target').count()['diagnosis'].to_list()
target_0 = targets[0]
target_1 = targets[1]
total = target_0 + target_1

class_weight_0 = (1 / target_0) * (total) / 2.0
class_weight_1 = (1 / target_1) * (total) / 2.0

class_weights = {0: class_weight_0, 1: class_weight_1}

initial_bias = np.log([target_1 / target_0])

In [ ]:
with strategy.scope():
    model = create_model_pretrained(bias_output=initial_bias)
    model.summary()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_auc', #val_auc
                                patience=25,
                                mode='max',
                                restore_best_weights=True)

def lr_schedule_fn(epoch, lr):
    if epoch < 8:
        return 0.000001
    elif epoch == 8:
        return 0.001
    elif epoch %2 ==0 and epoch < 49:
        return lr * 0.75
    else:
        return lr
lr_schedule = tf.keras.callbacks.LearningRateScheduler(lr_schedule_fn)


history = model.fit(
    train_ds,
    epochs= params['epochs'], 
    steps_per_epoch=epoch_steps,
    validation_data=valid_ds,
    validation_steps=valid_steps,
    class_weight=class_weights,
    callbacks=[early_stopping, lr_schedule],
    verbose = 0
)

In [ ]:
predictions = model.predict(test_ds.map(lambda img, igs: img), steps=test_steps)


In [ ]:
prediction_ids = next(iter(test_ds.
                          map(lambda img, ids:ids).
                          unbatch().
                          batch(test_size))).numpy().astype('str')

In [ ]:
prediction_dict = {
    'image_name': prediction_ids,
    'target': np.concatenate(predictions)
}
submission_ds = pd.DataFrame(prediction_dict)

In [ ]:
submission_ds.head(1)

In [ ]:
submission_ds.to_csv('submission.csv', index=False)

In [ ]:
def plot_metric(history_, metric1, metric2, ylabel):
    plt.plot(history_.history[metric1], label=metric1)
    plt.plot(history_.history[metric2], label=metric2)
    plt.ylabel(ylabel)
    plt.xlabel('epoch')
    plt.legend()
    plt.show()


In [ ]:
plot_metric(history, 'auc', 'val_auc', 'auc')
plot_metric(history, 'loss', 'val_loss', 'loss')
plot_metric(history, 'accuracy', 'val_accuracy', 'accuracy')

In [ ]:
history.history